In [3]:
from openvqe.vqe import VQE

In [4]:
# user can type the name of molecule (H2, LIH, CO, CO2 such that their geormetries and properties are defined in MoleculeQlm())
molecule_symbol = 'H2'
# the type of generators: UCCSD, singlet_sd, singlet_gsd, spin_complement_gsd, spin_complement_gsd_twin, sUPCCGSD
# suppose user type sUPCCGSD
type_of_generator = 'sUPCCGSD'
# user can type any of the following three transformations: JW,  Bravyi-Kitaev and Parity-basis
transform = 'JW'
# the non_active space selection
active = False

In [5]:
VQE('ucc', molecule_symbol, type_of_generator, transform, active).execute()

 --------------------------------------------------------------------------
Running in the non active case: 
 molecule symbol: H2 
molecule basis: 6-31g 
type of generator: sUPCCGSD 
transform: JW 
options: {'step': 0.01} 
 --------------------------------------------------------------------------
 --------------------------------------------------------------------------
                                                          
                      Generate Hamiltonians and Properties from :
                                                          
 --------------------------------------------------------------------------
                                                          
Number of electrons =  2
Number of qubits before active space selection =  8
Orbital energies =  [-0.59279976  0.23579258  0.77956294  1.39138474]
Nuclear repulsion =  0.70556961456
Noons =  [1.9705947706867002, 0.024127481185769732, 0.005019126551194664, 0.0002586215763354261]
 --------------------------

/home/nathan/miniconda/envs/openvqe/lib/python3.11/site-packages/openvqe/common_files/molecule_factory.py:473: UserWarning: This guess_init_state function is deprecated.
  _, _, hf_init = get_cluster_ops_and_init_guess(
/home/nathan/miniconda/envs/openvqe/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Optimization terminated successfully.
         Current function value: -1.151689
         Iterations: 32
         Function evaluations: 760
         Gradient evaluations: 40
Optimization terminated successfully.
         Current function value: -1.143323
         Iterations: 11
         Function evaluations: 418
         Gradient evaluations: 22
iterations are: {'minimum_energy_result1_guess': [-1.1516885455361772], 'minimum_energy_result2_guess': [-1.1433228515740375], 'theta_optimized_result1': [[-0.0014613903866998765, -0.004821433493245651, 0.009369016026938766, 0.004314550376230582, 0.008378591527264985, 0.011600224457440605, -0.10858256682196482, -0.05255181285793247, -0.015314630835684954, -0.02023926667795745, -0.03138182321349586, -0.007692596772005485, 0.0013326441307544422, -0.0011437511656627309, -0.00041316814693899136, -0.0043646266976174715, -0.6003496849563118, -0.0002346915874939363]], 'theta_optimized_result2': [[1.5326108250285396e-05, 1.9948836799260614e-05, -0.0014